In [1]:
def readFastq(filename):
    sequences = []
    with open(filename) as fh:
        while True:
            fh.readline() # skip name line
            seq = fh.readline().rstrip() # read base sequence
            fh.readline() # skip placeholder line
            fh.readline() # skip base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
    return sequences

In [2]:
chr1_raw = open('chr1.GRCh38.excerpt.fasta')

In [3]:
count = 0
for line in chr1_raw:
#     print(line.strip())
    count += 1
print(count)

10001


In [4]:
chr1_raw = open('chr1.GRCh38.excerpt.fasta')

In [5]:
chr1_reads = [line.strip() for line in chr1_raw]

In [6]:
chr1_reads[:10]

['>CM000663.2_excerpt EXCERPT FROM CM000663.2 Homo sapiens chromosome 1, GRCh38 reference primary assembly',
 'TTGAATGCTGAAATCAGCAGGTAATATATGATAATAGAGAAAGCTATCCCGAAGGTGCATAGGTCAACAATACTTGAGCC',
 'TAACTCAGTAGATCCTAAAAGAAAGCAATTTTTGCTGCTAACCTAACATTTCACAATGTCTGGAGACATTTACAGTTCCC',
 'ACAACCTATGGCAGTTACTGGCATCTACTAGAGGTCAGAGATGCTGGTAAATACTCTGTAATGAACAAGAAGCCCCCCAT',
 'AGCAAATAAATACCCAGCCCAAGATGGCAATAGTGCCCAGATTGAGAAACTTCACCTTAACCTGATATCATGCAAATATA',
 'TCTGAAGAAAGACACAAACATAACTAAAGAAAGATGATTACCAGAAGAGATATTCATAAATCTTAGAAGCATAGAAAAAG',
 'AAACACAAGGCAATGTTTTCAGTGTCCAGGCAATTATCTTCCTGGGAAAAGCTAGCCTACCAGACCAACATGACTTTTGC',
 'ACCTTGCTGGCAACCATTCTACTCTTCTGAAGAAGGAGACATCATTTGGACTCTAAAATCCCTTTTTCTGATTTCATACT',
 'CATCAAGAAATCTATCCATTTGGCTTAGTTTGTAGCTTATGCTGAAAAACGTGACTTGAGATTTCCTTCACTTGGAAATT',
 'GAGATTGCTTAATGTAGATTGACATTCTCAACATTTGGACAATAGTGGGATCAATTATCTTAACTTGCAAAGCTGAAGAT']

In [7]:
chr1_reads = chr1_reads[1:] # drop id line

In [8]:
len(chr1_reads)

10000

In [9]:
chr1_seq = ''.join(chr1_reads) # combine all lines into single seq str

In [10]:
len(chr1_seq)

800000

In [11]:
chr1_seq[:100]

'TTGAATGCTGAAATCAGCAGGTAATATATGATAATAGAGAAAGCTATCCCGAAGGTGCATAGGTCAACAATACTTGAGCCTAACTCAGTAGATCCTAAAA'

# Practice Naive match alg w/o counts

In [12]:
def naive(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):
        match = True
        for j in range(len(p)):
            if t[i+j] != p[j]:
                match = False
                break
        if match:
            occurrences.append(i)
    return occurrences

In [13]:
# Example 1

p = 'word'
t = 'there would have been a time for such a word'
occurrences = naive(p, t)
print(occurrences)

[40]


In [14]:
# Example 2

p = 'needle'
t = 'needle need noodle needle'
occurrences = naive(p, t)
print(occurrences)

[0, 19]


In [15]:
def naive_with_counts(p, t):
    occurrences = []
    num_alignments = 0
    num_character_comparisons = 0
    for i in range(len(t) - len(p) + 1):
        match = True
        for j in range(len(p)):
            num_character_comparisons += 1
            if t[i+j] != p[j]:
                match = False
                break
        if match:
            occurrences.append(i)
        num_alignments += 1
    return occurrences, num_alignments, num_character_comparisons

# Implement Naive match alg with counts

In [16]:
# Naive
# Example 1

p = 'word'
t = 'there would have been a time for such a word'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

[40] 41 46


In [17]:
# Example 2

p = 'needle'
t = 'needle need noodle needle'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

[0, 19] 20 35


In [18]:
from bm_preproc import BoyerMoore

In [19]:
def boyer_moore(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences

# Practice boyer_moore match alg w/o counts

In [20]:
p = 'word'
t = 'there would have been a time for such a word'
lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences = boyer_moore(p, p_bm, t)
print(occurrences)

[40]


In [21]:
p = 'needle'
t = 'needle need noodle needle'
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences = boyer_moore(p, p_bm, t)
print(occurrences)

[0, 19]


In [22]:
def boyer_moore_with_counts(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    num_alignments = 0
    num_character_comparisons = 0
    while i < len(t) - len(p) + 1: # I think this loop is facilitating alignments?
        num_alignments += 1
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):  # Then this loop is doing character comparisons? Inshallah
            num_character_comparisons += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences, num_alignments, num_character_comparisons

In [23]:
p = 'word'
t = 'there would have been a time for such a word'
lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

[40] 12 15


In [24]:
p = 'needle'
t = 'needle need noodle needle'
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

[0, 19] 5 18


# Okay, enough practicing, LET'S CODE (actually do the hw quiz)

1.
Question 1
How many alignments does the naive exact matching algorithm try when matching the string GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [25]:
%time
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
t = chr1_seq
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 4.53 µs
[56922] 799954 984143


How many character comparisons does the naive exact matching algorithm try when matching the string GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

How many alignments does Boyer-Moore try when matching the string GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [26]:
%time
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
t = chr1_seq
p_bm = BoyerMoore(p)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 2.62 µs
[56922] 127974 165191


# Question 4 Index-assisted approximate matching.

How many times does the string GGCGCGGTGGCTCACGCCTGTAAT, which is derived from a human Alu sequence, occur with up to 2 substitutions in the excerpt of human chromosome 1?  (Don't consider reverse complements here.)

GGCGCGGTGGCTCACGCCTGTAAT

In [27]:
def naive_2mm(p, t):
    
    occurrences = []
    
    # loop over forward alignments
    for i in range(len(t) - len(p) + 1):  
        misses = 0
        match = True
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                misses += 1
            if misses >= 3:
                match = False
                break
#             print(j, misses)
        if match:
            occurrences.append(i)  # all chars matched; record
    
    return occurrences

In [28]:
%time
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
occurrences = naive_2mm(p, chr1_seq)
print(occurrences)
len(occurrences)

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 2.62 µs
[56922, 84641, 147558, 160162, 160729, 191452, 262042, 273669, 364263, 421221, 429299, 465647, 551134, 635931, 657496, 681737, 717706, 724927, 747359]


19

In [29]:
from kmer_index import Index

In [30]:
# instantiate index object for question #4 
index_4 = Index(chr1_seq ,8)

In [31]:
index_4.index[:10]

[('AAAAAAAA', 1867),
 ('AAAAAAAA', 1868),
 ('AAAAAAAA', 2631),
 ('AAAAAAAA', 2995),
 ('AAAAAAAA', 2996),
 ('AAAAAAAA', 2997),
 ('AAAAAAAA', 2998),
 ('AAAAAAAA', 2999),
 ('AAAAAAAA', 3000),
 ('AAAAAAAA', 3001)]

In [32]:
len(index_4.index)

799993

#5. Using the instructions given in Question 4, how many total index hits are there when searching for occurrences of GGCGCGGTGGCTCACGCCTGTAAT with up to 2 substitutions in the excerpt of human chromosome 1?

In [33]:
p = 'GGCGCGGTGGCTCACGCCTGTAAT'

In [36]:
len(index_4.query('AAAAAAAA'))

853

In [37]:
# 6 

In [38]:
import bisect
   
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits


In [39]:
ind = SubseqIndex('ATATAT', 3, 2)
print(ind.index)


[('AAA', 0), ('TTT', 1)]


In [41]:
10//5

2

In [42]:
def approximate_match_subseq_index(p, t, k):
    segment_length = round(len(p) // (k+1))
    hits = 0
    all_matches = set()
    index = SubseqIndex(t, 8, 3) # built on 8-mers and subsequence intervals of 3
    for start in range(k+1):
        matches = index.query(p[start:])
        hits += len(matches)
        for m in matches:
            text_offset = m - start
            if text_offset < 0 or (text_offset + len(p)) > len(t):
                continue
            mismatches = 0
            for j in range(0, len(p)):
                if not p[j] == t[text_offset + j]:
                    mismatches += 1
                    if mismatches > k:
                        break
            if mismatches <= k:
                all_matches.add(text_offset)
    return list(all_matches), hits

In [43]:
t = chr1_seq
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
k = 2
_, hits = approximate_match_subseq_index(p, t, k)
print(hits)

79


In [44]:
_[:10]

[84641, 160162, 724927, 273669, 147558, 364263, 421221, 681737, 717706, 465647]

In [ ]:
# double check with Boyer Moore
